In [ ]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

import requests, json
import pandas as pd

In [ ]:
# Carrega as variáveis do arquivo .env
load_dotenv()
database_url = os.getenv('DATABASE_URL')

# Faz conexão com Atlas
cliente = MongoClient(database_url)
db = cliente['projeto_fecomercio']

In [ ]:
colecao_ano_mes = [
    '202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012',
    '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', 
    '202201', '202202', '202203', '202204', '202205', '202206', '202207', '202208', '202209', '202210', '202211', '202212', 
    '202301', '202302', '202303', '202304', '202305', '202306', '202307', '202308', '202309', '202310', '202311', '202312',
    '202401', '202402', '202403', '202404', '202405', '202406', '202407', '202408', '202409', '202410'
]

anomes = colecao_ano_mes[0]

# Endepoints

url_pix_chaves = "https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata/ChavesPix(Data=@Data)?@Data='2020-01-01'"
url_pix_municipio = f"https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata/TransacoesPixPorMunicipio(DataBase=@DataBase)?@DataBase='{anomes}'"
url_pix_estatisticas = f"https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata/EstatisticasTransacoesPix(Database=@Database)?@Database='{anomes}'"

### pix - chave

In [ ]:
response_pix_chaves = requests(url_pix_chaves)

In [ ]:
if response_pix_chaves.status_code == 200:
    pix_chaves_data = response_pix_chaves.json()['value']
    print(pix_chaves_data)

    # cria um dataframe
    df_pix_chaves = pd.DataFrame(pix_chaves_data)
else:
    print("Erro:", response_pix_chaves.status_code, response_pix_chaves.text)

In [ ]:
collection_pix_chaves = db['pixChaves']
collection_pix_chaves.insert_many(df_pix_chaves.to_dict('records'))

### pix - municipios

In [ ]:
erros_pix_municipio = []
dados_pix_municipio = []
anomes = colecao_ano_mes[0]

for anomes in colecao_ano_mes:
    resposta = requests.get(f"https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata/TransacoesPixPorMunicipio(DataBase=@DataBase)?@DataBase='{anomes}'")
    if resposta.status_code == 200:
        dados = resposta.json()['value']
        for valor in dados:
            dados_pix_municipio.append({
                'ANO': int(str(valor['AnoMes'])[:4]),
                'MES': int(str(valor['AnoMes'])[4:]),
                'Municipio_Ibge': valor['Municipio_Ibge'],
                'Municipio': valor['Municipio'],
                'Estado_Ibge': valor['Estado_Ibge'],
                'Estado': valor['Estado'],
                'Sigla_Regiao': valor['Sigla_Regiao'],
                'Regiao': valor['Regiao'],
                'VL_PagadorPF': valor['VL_PagadorPF'],
                'QT_PagadorPF': valor['QT_PagadorPF'],
                'VL_PagadorPJ': valor['VL_PagadorPJ'],
                'QT_PagadorPJ': valor['QT_PagadorPJ'],
                'VL_RecebedorPF': valor['VL_RecebedorPF'],
                'QT_RecebedorPF': valor['QT_RecebedorPF'],
                'VL_RecebedorPJ': valor['VL_RecebedorPJ'],
                'QT_RecebedorPJ': valor['QT_RecebedorPJ'],
                'QT_PES_PagadorPF': valor['QT_PES_PagadorPF'],
                'QT_PES_PagadorPJ': valor['QT_PES_PagadorPJ'],
                'QT_PES_RecebedorPF': valor['QT_PES_RecebedorPF'],
                'QT_PES_RecebedorPJ': valor['QT_PES_RecebedorPJ']
            })
    else:
        erros_pix_municipio.append(anomes)

In [ ]:
erros_pix_municipio

In [ ]:
df_pix_municipio = pd.DataFrame(dados_pix_municipio)
collection_pix_municipio = db['pixMunicipio']
collection_pix_municipio.insert_many(df_pix_municipio.to_dict('records'))

### pix - estatisticas

In [ ]:
erros_pix_estatisticas = []
dados_pix_estatisticas = []
anomes = colecao_ano_mes[0]

for anomes in colecao_ano_mes:
    resposta = requests.get(f"https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata/EstatisticasTransacoesPix(Database=@Database)?@Database='{anomes}'")
    if resposta.status_code == 200:
        dados = resposta.json()['value']
        for valor in dados:
            dados_pix_estatisticas.append({
                'ANO': int(str(valor['AnoMes'])[:4]),
                'MES': int(str(valor['AnoMes'])[4:]),
                'PAG_PFPJ': valor['PAG_PFPJ'],
                'REC_PFPJ': valor['REC_PFPJ'],
                'PAG_REGIAO': valor['PAG_REGIAO'],
                'REC_REGIAO': valor['REC_REGIAO'],
                'PAG_IDADE': valor['PAG_IDADE'],
                'REC_IDADE': valor['REC_IDADE'],
                'FORMAINICIACAO': valor['FORMAINICIACAO'],
                'NATUREZA': valor['NATUREZA'],
                'FINALIDADE': valor['FINALIDADE'],
                'VALOR': valor['VALOR'],
                'QUANTIDADE': valor['QUANTIDADE']
            })
    else:
        erros_pix_estatisticas.append(anomes)

In [ ]:
erros_pix_estatisticas

In [ ]:
df_pix_estatisticas = pd.DataFrame(dados_pix_estatisticas)
collection_pix_estatisticas = db['pixEstatisticas']
collection_pix_estatisticas.insert_many(df_pix_estatisticas.to_dict('records'))